BOUZARKOUNA Malek 28706508
BABOYAN Sevag 3521887
# TME 3.2: Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm random Projections.


1\.  Nous allons réutiliser les fonctions du precedent pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [213]:
import random
import numpy as np

nuc = ('A', 'C', 'G', 'T')

t=5 #nombre de sequences
v=1 #nb de positions variable dans le motif
k=3 #taille du motif
n=10 #longuer des sequence
f= 0.5 #frequence de variation du motif.


In [214]:
def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Génère une séquence nucléotidique aléatoire 
    entrée n : longueur de la sequence
    entrée upper : bool, si True les nucléotides seront minuscule, False majuscule
    sortie sequence : une séquence nucléotidique aléatoire 
    """
    sequence = ""
    for i in range(n):
        sequence += random.choice(nuc) if upper else random.choice(nuc).lower()
    return sequence

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)
    for i in range(nbpos):
        motifM[random.randint(0,len(motifM)-1)]= random.choice(nuc)
    
    return "".join(motifM).upper() if upper else "".join(motifM).lower()
#tester modifMotif
print (modifierMotif("acg", 2))


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : longueur de séquences 
    entrée n : nombre des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, 
    pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """
    
    motif = generateRandomSequence(t)
    print(motif)
    sequences = [generateRandomSequence(n - t) for i in range(k)]

    for i in range(k):
        newmotif = motif
        if (random.random() < f):
            newmotif = modifierMotif(motif, v)
        sequences[i] = insertMotif(sequences[i],newmotif , random.randint(0, t - k))
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

cca
tggag
['ttggagaatc', 'ttgaaggtca', 'tgggggggct']
['TTGGAGAATC', 'TTGAAGGTCA', 'TGGGGGGGCT']


2\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de p à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [215]:
def getRandomFixePositions(k, p):
    """
    Genere une projection de p vers k
    entrée p: nombre de positions choisi pour la projection 
    entrée k: nombre de positions du motif original
    sortie projection: liste de positions choisi aléatoirement
    """
    #projection = random
    #count = 0
    return random.sample(range(k), p)
#print(getRandomFixePositions(10,10))

lR = getRandomFixePositions(7, 4)
print(lR)
lR.sort()
print (lR)

def generateKey(projection, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée projection : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    cle = ""; j = 0; i = 0
    
    while i < len(motif) :
        while j < len(projection) :
            if (i == projection[j]) :
                #print("motif: ",motif)
                #print("len m: ", len(motif))
                #print("i,j = ",i,j)
                #print("dist", motif[i])
                cle += motif[i]
                j += 1
            else :
                cle += "-"
            if (i+1 == len(motif)):
                break
            i += 1
        cle += "-"
        i += 1
    return cle

generateKey(lR, "0123456789")

[2, 1, 3, 6]
[1, 2, 3, 6]


'-123--6---'

3\. Implémenter l'algorithme ``randomProjection``. Bonnus : Pour ameliorer la performance vous pouvez abandonner les motifs de taille k peu complexes.

In [216]:
def lowComplex(motif, minrep):
    double_consecutif = 0
    for i in range(0,len(motif)-3):
        if (motif[i] == motif[i+2] and motif[i+1] == motif[i+3]):
            double_consecutif = 1
            return False
    if ((double_consecutif == 1) or ((motif.lower()).count('a') >=  minrep) or  ((motif.lower()).count('t') >=  minrep) or  ((motif.lower()).count('c') >=  minrep) or  ((motif.lower()).count('g') >=  minrep)):
        return False
    return True
    

def randomProjection(k, p, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : taille du motif
    entrée p : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur= original motif
    """
    motifs  = {}; motifsSeq  = {}
    longueur = len(sequences[0])
    projection = getRandomFixePositions(k,p)
    #print(projection)
    projection.sort()

    for sequence in sequences:

        for i in range(longueur - k + 1):#+1
            
            seq = sequence[i:i+k]
            genkey = generateKey(projection, seq)

            if(lowComplex(genkey, 4)):
                if genkey in motifs :
                    motifs[genkey] += 1
                else :
                    motifs[genkey] = 1
                
                if genkey in motifsSeq :
                    motifsSeq[genkey].append(seq)
                else :
                    motifsSeq[genkey] = [seq]
            
    
    return motifs, motifsSeq, projection

#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq, proj = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)
print (proj)
#motifsProj => {'TACG': 1, 'TCGC': 3, 'AGGA': 1, 'AGCC': 1, 'GCAG': 1, 'CACC': 1, 'CGCA': 1, 'AGGC': 1, 'CGCG': 1, 'CCCT': 1}
#motifsSeq => {'TACG': ['TTAACGG'], 'TCGC': ['TAACGGC', 'TCACGCC', 'TACCGGC'], 'AGGA': ['AACGGCA'], 'AGCC': ['ACGGCAC'], 'GCAG': ['GCTCACG'], 'CACC': ['CTCACGC'], 'CGCA': ['CACGCCA'], 'AGGC': ['ACCGGCC'], 'CGCG': ['CCGGCCG'], 'CCCT': ['CGGCCGT']}

{'TT--CG-': 1, 'TA--GG-': 2, 'AA--GC-': 1, 'AC--CA-': 1, 'GC--AC-': 1, 'CT--CG-': 1, 'TC--GC-': 1, 'CA--CC-': 1, 'AC--GC-': 1, 'CG--CG-': 1}
{'TT--CG-': ['TTAACGG'], 'TA--GG-': ['TAACGGC', 'TACCGGC'], 'AA--GC-': ['AACGGCA'], 'AC--CA-': ['ACGGCAC'], 'GC--AC-': ['GCTCACG'], 'CT--CG-': ['CTCACGC'], 'TC--GC-': ['TCACGCC'], 'CA--CC-': ['CACGCCA'], 'AC--GC-': ['ACCGGCC'], 'CG--CG-': ['CGGCCGT']}
[0, 1, 4, 5]


4\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

reponse: 

5\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [217]:

#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    q = len(nuc)
    PWM = np.zeros((q, k))
    
    for s in motifs :
        for j in range(k) :
            if len(s) < k :
                continue
            l = s[j]
            if l == 'A':
                PWM[0][j] += 1
            elif l == 'C':
                PWM[1][j] += 1
            elif l == 'G':
                PWM[2][j] += 1
            elif l == 'T':
                PWM[3][j] += 1
    return PWM
  

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    sc = 0
    
    t = len(MF)
    n = len(MF[0])
    m = 0
    for i in range(n) :
        m = 0
        for j in range(t) :
            val = MF[j][i]
            if val > m :
                m = val
        sc += m
    
    return sc

In [218]:
def randomProjIt(sequences, k, v, nuc, It):
    """
    Implémente l'algorithme randomProjection version iteractive
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
 
    scores = 0; dico = {}; bp = ""; bm = []
    
    for i in range(It):
        motifsProj, motifsSeq, gk = randomProjection(k, v, sequences)
        tmp_l = list(motifsProj.keys())
        for j in tmp_l:
            msj = motifsSeq[j]
            tmp_pr = profile(msj, k, nuc)
            tmp = getScore(tmp_pr, k)
            if(tmp > scores):
                scores = tmp
                bp = j
                bm = msj
        dico[bp] = bm
                
    return scores, tmp_l , dico, bp, bm

score, seqsMotif, dico, bp, bm = randomProjIt(adnTest, 8, 4, nuc, 100)

print (score, seqsMotif, bp, bm)


20.0 ['TT--C-G-', 'TA--G-C-', 'AA--G-A-', 'GC--A-G-', 'TC--G-C-', 'AC--G-C-', 'CC--C-G-'] T--CG-C- ['TAACGGCA', 'TCACGCCA', 'TACCGGCC']


6\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [219]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in seq])

def printTopFMotifsFreq(motifs, m, rev=False):
    motifsRet = {}
    motifsSort = sorted(motifs, reverse=True, key=motifs.get)
    i = 0
    while (i < m):
        motifPrint = motifsSort[i]
        print (motifsSort[i])
        if rev:
            motifPrint = reverseComplement(motifsSort[i])
        print (i, motifPrint, "-", motifs[motifsSort[i]])
        i = i + 1
        
#printTopFMotifsFreq(motifs, 10)
#print (motifsSeq['TCGC'])

In [220]:
k=8; p=4; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_3.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip) - 2
print (sequencesChip[8], t, n, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequences = sequencesChip + revSequences

score, seqsMotif, dico, bp, bm = randomProjIt(sequences, k, p, nuc, 50)
#print(score, seqsMotif)
print(score)
print(dico)

def logofile(motif_occur : dict):
    """cette fonction sert à transfromer le dictionnaire en un fichier qu'on peut imporeter dans le site LOGO"""
    fichier = open("dataLogo.txt", "w+")
    for i in motif_occur :
        for j in range(motif_occur[i]):
            fichier.write(i+'\n')
    fichier.close()
#logofile(motif_occur_sorted)


#tab_consensus = {'TTTGCTCA':0, 'CTCATTGG':0, 'CTGATTGG':0, 'TCCGCGGA':0, 'TCCGTGGA':0, 'CAGCAAAA':0, 'CGGATCCG':0 }
##tab_consensus = {'CCTTNAGGGT':0}
##tab_consensus = {'TGCACCC':0}
##tab_consensus = {'CAGCAA':0}
##tab_consensus = {'C':0}
#tab_cpt =[0,0,0,0,0,0,0]
#for truc in seqsMotif:
#    if truc in tab_consensus.keys():
#        print(truc)
#        tab_consensus[truc] += 1
#print(tab_consensus)
#print(tab_consensus.keys())

AAGTTGTAGTACTTGTGTTCCTGGTTTGCTGGTTTGCTTGGTTTGTCGGTGTATCTGCAGTTGTGTATTTGCAGGTCTGT 575 80 8
4301.0
{'-A-T--TT-': ['AACTCCTT', 'TAGTACTT', 'AAATTTTT', 'AATTTTTT', 'AATTTATT', 'GACTTGTT', 'GATTGCTT', 'GACTTGTT', 'TATTATTT', 'AAATTTTT', 'TAATGTTT', 'TAATAGTT', 'TAGTTTTT', 'TAATTTTT', 'TATTGATT', 'AAATTTTT', 'TATTGGTT', 'TAATCATT', 'GATTTGTT', 'CATTCTTT', 'TATTCCTT', 'TAGTATTT', 'CAATATTT', 'TATTTCTT', 'TAGTCTTT', 'GAGTTATT', 'CATTTTTT', 'GAATGATT', 'CAGTTTTT', 'TATTCATT', 'CATTAGTT', 'TACTTATT', 'TATTGGTT', 'AAATCATT', 'AACTGATT', 'GAATAATT', 'CATTCATT', 'TATTCGTT', 'TATTGGTT', 'AAGTTTTT', 'GAATCTTT', 'TAGTTGTT', 'GAATCGTT', 'TAATTGTT', 'AATTCATT', 'CACTCATT', 'CAGTACTT', 'GAATCGTT', 'AATTGCTT', 'CAGTTTTT', 'TATTATTT', 'AAGTTTTT', 'TATTTTTT', 'CAGTTTTT', 'CAATTATT', 'AATTATTT', 'TATTTTTT', 'TAATTCTT', 'AATTCTTT', 'GAATGGTT', 'TACTCTTT', 'TAGTACTT', 'CATTTTTT', 'TACTATTT', 'AAATAATT', 'GAGTATTT', 'TATTTATT', 'CAATGGTT', 'CACTACTT', 'AAGTGATT', 'CACTGTTT', 'GAGTTTTT', 'AACTGATT', 'GATTTCTT', 

In [221]:
print(bp)
for i in dico[bp]:
    print(i)

T-TAT---
TATATAGC
TTTATACT
TGTATCTG
TGTATCTG
TGTATTTG
TGTATCCC
TGTATCCC
TGTATCAG
TATATAAA
TTTATTTG
TTTATGAG
TGTATAAT
TTTATTGA
TCTATCTT
TGTATATC
TATATCTT
TTTATCTA
TCTATTGT
TGTATTAT
TGTATTCG
TGTATTAC
TATATTAT
TGTATTAC
TATATATC
TATATCCT
TATATGTA
TATATACA
TGTATCCC
TGTATGTA
TATATATA
TATATAAT
TCTATGTC
TCTATTCA
TCTATTTG
TATATTCT
TTTATTTA
TGTATATA
TATATATT
TATATTGA
TATATAAT
TATATATC
TATATCGA
TATATATT
TATATTGT
TGTATACC
TATATTGG
TGTATATT
TATATTAT
TGTATGAT
TCTATTCA
TGTATAAT
TTTATAGT
TGTATACA
TTTATATG
TATATGAA
TCTATCTG
TGTATATA
TATATATA
TATATATA
TATATATT
TATATTTA
TTTATCAT
TGTATGAC
TCTATCAG
TCTATCCT
TATATCAT
TGTATGAC
TGTATTTC
TGTATCTC
TCTATAAC
TTTATTTT
TTTATTTT
TGTATATA
TATATAAT
TTTATATA
TATATATA
TATATAAA
TTTATCTT
TTTATTCG
TGTATACT
TGTATCGG
TCTATCTA
TTTATTTT
TTTATGAT
TCTATAAT
TATATACA
TGTATACT
TGTATAGT
TATATATT
TATATTAT
TGTATGCT
TGTATACT
TATATGTG
TTTATACT
TTTATGGC
TGTATAAC
TTTATGTG
TATATAAC
TATATTGT
TTTATATG
TATATGAT
TCTATTTG
TCTATGAA
TGTATAAT
TCTATTGG
TATATTAT
TGTATTAG
TATATACA
TATATACT
TATATCAC
T